In [ ]:
import { load } from "dotenv";
const env = await load({
  envPath:'./.env.local',
  export: true
});

const process = {
    env
}

const chatOptions = {
  openAIApiKey: process.env.OPENAI_API_KEY,
  temperature: 1.5,
  model: "deepseek-chat",
  configuration: {
    baseURL: "https://api.deepseek.com",
  },
  azureOpenAIBasePath: "https://api.deepseek.com",
}

console.log(process.env)

In [ ]:
import { PromptTemplate } from "@langchain/core/prompts";

const personalizedGreetingPrompt = new PromptTemplate({
  inputVariables: ["name"],
  template: "hello，{name}",
});
const formattedPersonalizedGreeting = await personalizedGreetingPrompt.format({
  name: "Ginlon",
});

console.log(formattedPersonalizedGreeting);

简化写法

In [ ]:
import { PromptTemplate } from "@langchain/core/prompts";

const personalizedGreetingPrompt = PromptTemplate.fromTemplate("good {timeOfDay}, {name}");
const formattedPersonalizedGreeting = await personalizedGreetingPrompt.format({
  timeOfDay: "morning",
  name: "Ginlon",
});

console.log(formattedPersonalizedGreeting);

柯里化输入

模板所需要的参数可以部分输入，每次输入一部分参数都会创建一个新的 PromptTemplate 实例，新的实例会继承之前实例的参数。

In [ ]:
const initialPrompt = new PromptTemplate({
  template: "这是一个{type}，它是{item}。",
  inputVariables: ["type", "item"],
});


const partialPrompt = await initialPrompt.partial({
  type: "工具",
});

const formattedPrompt = await partialPrompt.format({
  item: "锤子",
});

console.log(formattedPrompt);
// 这是一个工具，它是锤子。

const formattedPrompt2 = await partialPrompt.format({
  item: "改锥",
});

console.log(formattedPrompt2)
// 这是一个工具，它是改锥。


动态填充参数

In [ ]:
const getCurrentDateStr = () => {
  return new Date().toLocaleDateString();
};

function generateGreeting(timeOfDay) {
  return () => {
    const date = getCurrentDateStr()
    switch (timeOfDay) {
      case 'morning':
        return date + ' 早上好';
      case 'afternoon':
        return date + ' 下午好';
      case 'evening':
        return date + ' 晚上好';
      default:
        return date + ' 你好';
    }
  };
}

const prompt = new PromptTemplate({
  template: "{greeting}!",
  inputVariables: ["greeting"],
});

const currentTimeOfDay = 'afternoon';
const partialPrompt = await prompt.partial({
  greeting: generateGreeting(currentTimeOfDay),
});

const formattedPrompt = await partialPrompt.format();

console.log(formattedPrompt);

## 消息角色

为了方便地构建和处理结构化的聊天消息，LangChain 提供了几种与聊天相关的提示模板类，如 `ChatPromptTemplate``、SystemMessagePromptTemplate`、`AIMessagePromptTemplate` 和 `HumanMessagePromptTemplate`

- system 角色的消息通常用于设置对话的上下文或指定模型采取特定的行为模式。这些消息不会直接显示在对话中，但它们对模型的行为有指导作用。 可以理解成模型的元信息，权重非常高，在这里有效的构建 prompt 能取得非常好的效果。  
- user 角色代表真实用户在对话中的发言。这些消息通常是问题、指令或者评论，反映了用户的意图和需求。  
- assistant 角色的消息代表AI模型的回复。这些消息是模型根据system的指示和user的输入生成的。  

In [ ]:
import {
  SystemMessagePromptTemplate,
  HumanMessagePromptTemplate,
  ChatPromptTemplate
} from "@langchain/core/prompts"

const translateInstructionTemplate =
  SystemMessagePromptTemplate.fromTemplate(`你是一个专
业的翻译员，你的任务是将文本从{source_lang}翻译成{target_lang}。`)

const userQuestionTemplate =
  HumanMessagePromptTemplate.fromTemplate("请翻译这句话：{text}")

const chatPrompt = ChatPromptTemplate.fromMessages([
  translateInstructionTemplate,
  userQuestionTemplate,
]);

const formattedChatPrompt = await chatPrompt.formatMessages({
  source_lang: "中文",
  target_lang: "法语",
  text: "你好，世界",
});

console.log(formattedChatPrompt)

简化语法

In [ ]:
import { ChatPromptTemplate } from "@langchain/core/prompts"

const systemTemplate =
  "你是一个专业的翻译员，你的任务是将文本从{source_lang}翻译成{target_lang}。"
const humanTemplate = "请翻译这句话：{text}"

const chatPrompt = ChatPromptTemplate.fromMessages([
  ["system", systemTemplate],
  ["human", humanTemplate],
])

const formatPrompt = await chatPrompt.formatMessages({
  source_lang: "中文",
  target_lang: "法语",
  text: "你好，世界",
})

console.log(formatPrompt)


In [ ]:
import { ChatOpenAI } from '@langchain/openai' 
import { StringOutputParser } from "@langchain/core/output_parsers";

const chatModel = new ChatOpenAI(chatOptions);
const outputParser = new StringOutputParser();

const chain = chatPrompt.pipe(chatModel).pipe(outputParser);

await chain.invoke({
  source_lang: "中文",
  target_lang: "法语",
  text: "你好，世界",
})

## 组合 Prompt

可以用将多个独立的 template 构建成一个完整且复杂的 prompt

`PipelinePromptTemplate` 有两个核心的概念：

pipelinePrompts，一组 object，每个 object 表示 prompt 运行后赋值给 name 变量
finalPrompt，表示最终输出的 prompt

In [ ]:
import {
  PromptTemplate,
  PipelinePromptTemplate,
} from "@langchain/core/prompts";

const getCurrentDateStr = () => {
  return new Date().toLocaleDateString();
};

const fullPrompt = PromptTemplate.fromTemplate(`
你是一个智能管家，今天是 {date}，你的主人的信息是{info}, 
根据上下文，完成主人的需求
{task}`);

const datePrompt = PromptTemplate.fromTemplate("{date}，现在是 {period}")
const periodPrompt = await datePrompt.partial({
    date: getCurrentDateStr
})

const infoPrompt =  PromptTemplate.fromTemplate("姓名是 {name}, 性别是 {gender}");

const taskPrompt = PromptTemplate.fromTemplate(`
我想吃 {period} 的 {food}。 
再重复一遍我的信息 {info}`);

const composedPrompt = new PipelinePromptTemplate({
  pipelinePrompts: [
    {
      name: "date",
      prompt: periodPrompt,
    },
    {
      name: "info",
      prompt: infoPrompt,
    },
    {
      name: "task",
      prompt: taskPrompt,
    },
  ],
  finalPrompt: fullPrompt,
});

const formattedPrompt = await composedPrompt.format({
    period: "早上",
    name: "张三",
    gender: "male",
    food: "lemon"
});

console.log(formattedPrompt)


langchain 会自动分析 pipeline 之间的依赖关系，尽可能的进行并行化来提高运行速度